In [42]:
import pandas as pd

# Load the dataset
df = pd.read_csv("dataset_part_2.csv")

# Print all column names
print("Column names in dataset_part_2:")
print(df.columns.tolist())


Column names in dataset_part_2:
['Flight No.', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Version Booster', 'Booster landing', 'Date', 'Time', 'Class']


In [43]:
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

# Load data
spacex_df = pd.read_csv("dataset_part_2.csv")

# Create list of launch sites for dropdown
launch_sites = [{'label': 'All Sites', 'value': 'ALL'}] + \
    [{'label': site, 'value': site} for site in spacex_df['Launch site'].unique()]

# Initialize Dash app
app = Dash(__name__)

# Layout with dropdown
app.layout = html.Div([
    html.H1("SpaceX Launch Dashboard", style={'textAlign': 'center'}),
    dcc.Dropdown(
        id='site-dropdown',
        options=launch_sites,
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True
    ),
    html.Br(),
    html.Div(dcc.Graph(id='success-pie-chart'))
])

# Run the app

if __name__ == '__main__':
    app.run(debug=True, port=8051)




In [44]:
# Task 2: Callback to render pie chart based on selected site
from dash import Dash, dcc, html, Input, Output
import pandas as pd
import plotly.express as px

# Load data
spacex_df = pd.read_csv("dataset_part_2.csv")

# Launch site options
launch_sites = [{'label': 'All Sites', 'value': 'ALL'}] + \
               [{'label': site, 'value': site} for site in spacex_df['Launch site'].unique()]

# Initialize app
app = Dash(__name__)

# App layout
app.layout = html.Div([
    html.H1('SpaceX Launch Dashboard', style={'textAlign': 'center'}),
    dcc.Dropdown(
        id='site-dropdown',
        options=launch_sites,
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True
    ),
    html.Br(),
    dcc.Graph(id='success-pie-chart')
])

# Callback to update pie chart
@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown', 'value')
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        fig = px.pie(
            spacex_df,
            names='Launch site',
            values='Class',
            title='Total Successful Launches by Site'
        )
    else:
        filtered_df = spacex_df[spacex_df['Launch site'] == selected_site]
        success_count = filtered_df['Class'].value_counts().reset_index()
        success_count.columns = ['Outcome', 'Count']
        success_count['Outcome'] = success_count['Outcome'].replace({1: 'Success', 0: 'Failure'})

        fig = px.pie(
            success_count,
            names='Outcome',
            values='Count',
            title=f'Success vs Failure for site {selected_site}'
        )
    return fig

# Run the app
app.run(port=8051, debug=False)



In [45]:
# Clean the 'Payload mass' column
spacex_df['Payload mass'] = spacex_df['Payload mass'].astype(str)  # Convert to string
spacex_df['Payload mass'] = spacex_df['Payload mass'].str.replace('[^0-9.]', '', regex=True)
spacex_df['Payload mass'] = pd.to_numeric(spacex_df['Payload mass'], errors='coerce')  # Convert to numeric


In [46]:
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

# Load the dataset
spacex_df = pd.read_csv("dataset_part_2.csv")

# Clean Payload mass column (in case it has strings like '~5,000 kg')
if not pd.api.types.is_numeric_dtype(spacex_df['Payload mass']):
    spacex_df['Payload mass'] = spacex_df['Payload mass'].astype(str).str.replace('[^0-9.]', '', regex=True)
    spacex_df['Payload mass'] = pd.to_numeric(spacex_df['Payload mass'], errors='coerce')

# Drop rows with missing payload mass
spacex_df = spacex_df.dropna(subset=['Payload mass'])

# List of launch sites for dropdown
launch_sites = [{'label': 'All Sites', 'value': 'ALL'}] + \
               [{'label': site, 'value': site} for site in spacex_df['Launch site'].unique()]

# Get min/max payload for the slider
min_payload = int(spacex_df['Payload mass'].min())
max_payload = int(spacex_df['Payload mass'].max())

# Initialize the app
app = Dash(__name__)

# App layout including Payload RangeSlider
app.layout = html.Div([
    html.H1('SpaceX Launch Dashboard', style={'textAlign': 'center'}),

    # Dropdown
    dcc.Dropdown(
        id='site-dropdown',
        options=launch_sites,
        value='ALL',
        placeholder="Select a Launch Site here",
        searchable=True
    ),

    html.Br(),

    # Pie Chart placeholder
    dcc.Graph(id='success-pie-chart'),

    html.Br(),

    # Payload Range Slider
    html.P("Payload range (Kg):"),
    dcc.RangeSlider(
        id='payload-slider',
        min=min_payload,
        max=max_payload,
        step=100,
        value=[min_payload, max_payload],
        marks={
            min_payload: str(min_payload),
            max_payload: str(max_payload)
        }
    ),

    html.Br(),

    # Placeholder for scatter chart (will be completed in Task 4)
    dcc.Graph(id='success-payload-scatter-chart')
])

# Run the app
if __name__ == '__main__':
     app.run(port=8051, debug=True)


In [49]:
#TASK 4: Add a callback function to render the success-payload-scatter-chart scatter plot
@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown', 'value')
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        # Show total success counts for all sites
        fig = px.pie(
            spacex_df,
            names='Launch site',
            values='Class',
            title='Total Success Launches by Site'
        )
    else:
        # Show success vs. failure for the selected site
        filtered_df = spacex_df[spacex_df['Launch site'] == selected_site]
        fig = px.pie(
            filtered_df,
            names='Class',
            title=f'Total Success vs. Failure for site {selected_site}'
        )
    return fig
app.run(port=8051, debug=False)